# Anomaly Detection Model

## Skripsi - Hagen Kwik

## Setup & EDA

### Import Libraries

In [60]:
# library for mathematical functions and data preprocessing such as table, etc
import pandas as pd
import numpy as np

# Library for preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

# Library for machine learning functions/algorithms
from sklearn.cluster import DBSCAN
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Dropout
from keras.optimizers import SGD, Nadam
from keras.optimizers.legacy import Adam

# Use the legacy Adam optimizer
from mealpy import WOA, DE
from mealpy.utils.problem import FloatVar
from sklearn.neighbors import NearestNeighbors

#  Library for evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

### KDD99 Dataset

In [61]:
# data
df_kdd99_training = pd.read_csv('kdd99-training.csv')
df_kdd99_training.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal


In [62]:
df_kdd99_testing = pd.read_csv('kdd99-testing.csv')
df_kdd99_testing.head()


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.03,0.03,back.
1,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.01,0.01,0.04,0.04,back.
2,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.03,0.03,back.
3,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.00,0.00,0.02,0.02,back.
4,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.0,0.0,0.0,0.0,0.01,0.01,0.05,0.05,back.


In [63]:
df_kdd99_training.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,494020.000000,4.940200e+05,4.940200e+05,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,...,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000,494020.000000
mean,47.979400,3.025616e+03,8.685308e+02,0.000045,0.006433,0.000014,0.034519,0.000152,0.148245,0.010212,...,232.471248,188.666052,0.753781,0.030906,0.601936,0.006684,0.176754,0.176443,0.058118,0.057412
std,707.747185,9.882191e+05,3.304003e+04,0.006673,0.134805,0.005510,0.782103,0.015520,0.355343,1.798328,...,64.744601,106.040205,0.410780,0.109259,0.481309,0.042133,0.380593,0.380920,0.230590,0.230141
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,4.500000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,46.000000,0.410000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,5.200000e+02,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.032000e+03,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,255.000000,1.000000,0.040000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,884.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [64]:
df_kdd99_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494020 entries, 0 to 494019
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     494020 non-null  int64  
 1   protocol_type                494020 non-null  object 
 2   service                      494020 non-null  object 
 3   flag                         494020 non-null  object 
 4   src_bytes                    494020 non-null  int64  
 5   dst_bytes                    494020 non-null  int64  
 6   land                         494020 non-null  int64  
 7   wrong_fragment               494020 non-null  int64  
 8   urgent                       494020 non-null  int64  
 9   hot                          494020 non-null  int64  
 10  num_failed_logins            494020 non-null  int64  
 11  logged_in                    494020 non-null  int64  
 12  num_compromised              494020 non-null  int64  
 13 

In [65]:
df_kdd99_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 311028 entries, 0 to 311027
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     311028 non-null  int64  
 1   protocol_type                311028 non-null  object 
 2   service                      311028 non-null  object 
 3   flag                         311028 non-null  object 
 4   src_bytes                    311028 non-null  int64  
 5   dst_bytes                    311028 non-null  int64  
 6   land                         311028 non-null  int64  
 7   wrong_fragment               311028 non-null  int64  
 8   urgent                       311028 non-null  int64  
 9   hot                          311028 non-null  int64  
 10  num_failed_logins            311028 non-null  int64  
 11  logged_in                    311028 non-null  int64  
 12  num_compromised              311028 non-null  int64  
 13 

In [66]:
# Check which column needs fixing
for column in df_kdd99_testing.columns:
    unique_values = df_kdd99_testing[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'duration':
[    0     9     2 ...  5049 10121  5033]

Unique values in 'protocol_type':
['tcp' 'icmp' 'udp']

Unique values in 'service':
['http' 'telnet' 'login' 'imap4' 'eco_i' 'private' 'smtp' 'whois' 'ssh'
 'ctf' 'ecr_i' 'domain' 'ftp' 'gopher' 'finger' 'rje' 'time' 'uucp' 'efs'
 'other' 'vmnet' 'klogin' 'netbios_ns' 'pop_2' 'netstat' 'remote_job'
 'shell' 'hostnames' 'ftp_data' 'http_443' 'netbios_ssn' 'iso_tsap'
 'csnet_ns' 'link' 'ldap' 'supdup' 'echo' 'pop_3' 'courier' 'name' 'nnsp'
 'exec' 'printer' 'systat' 'daytime' 'bgp' 'sql_net' 'auth' 'Z39_50' 'mtp'
 'kshell' 'uucp_path' 'netbios_dgm' 'sunrpc' 'nntp' 'discard' 'domain_u'
 'urp_i' 'IRC' 'ntp_u' 'urh_i' 'X11' 'tim_i' 'harvest']

Unique values in 'flag':
['SF' 'RSTR' 'RSTO' 'REJ' 'S0' 'SH' 'S1' 'S3' 'S2' 'OTH' 'RSTOS0']

Unique values in 'src_bytes':
[  54540   53168   54020 ...     520    1237 5133876]

Unique values in 'dst_bytes':
[     8314      7300      1460 ...      6944      4850 400291060]

Unique

In [67]:
# Check which column needs fixing
for column in df_kdd99_training.columns:
    unique_values = df_kdd99_training[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()

Unique values in 'duration':
[   0    1   79 ... 2695 2751  120]

Unique values in 'protocol_type':
['tcp' 'udp' 'icmp']

Unique values in 'service':
['http' 'smtp' 'finger' 'domain_u' 'auth' 'telnet' 'ftp' 'eco_i' 'ntp_u'
 'ecr_i' 'other' 'private' 'pop_3' 'ftp_data' 'rje' 'time' 'mtp' 'link'
 'remote_job' 'gopher' 'ssh' 'name' 'whois' 'domain' 'login' 'imap4'
 'daytime' 'ctf' 'nntp' 'shell' 'IRC' 'nnsp' 'http_443' 'exec' 'printer'
 'efs' 'courier' 'uucp' 'klogin' 'kshell' 'echo' 'discard' 'systat'
 'supdup' 'iso_tsap' 'hostnames' 'csnet_ns' 'pop_2' 'sunrpc' 'uucp_path'
 'netbios_ns' 'netbios_ssn' 'netbios_dgm' 'sql_net' 'vmnet' 'bgp' 'Z39_50'
 'ldap' 'netstat' 'urh_i' 'X11' 'urp_i' 'pm_dump' 'tftp_u' 'tim_i' 'red_i']

Unique values in 'flag':
['SF' 'S1' 'REJ' 'S2' 'S0' 'S3' 'RSTO' 'RSTR' 'RSTOS0' 'OTH' 'SH']

Unique values in 'src_bytes':
[181 239 235 ... 500 484 475]

Unique values in 'dst_bytes':
[ 5450   486  1337 ... 34557 10592 13828]

Unique values in 'land':
[0 1]

Unique valu

In [68]:
missing_values_kdd99_testing = df_kdd99_testing.isnull().sum()
print(missing_values_kdd99_testing)

duplicates_kdd99_testing = df_kdd99_testing.duplicated().sum()
print(duplicates_kdd99_testing)


duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

In [69]:
missing_values_kdd99_training = df_kdd99_training.isnull().sum()
print(missing_values_kdd99_training)

duplicates_kdd99_training = df_kdd99_training.duplicated().sum()
print(duplicates_kdd99_training)


duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_h

### UNSW-NB15 Dataset

In [70]:
# data
df_unsw_training = pd.read_csv('UNSW-training-set.csv')
df_unsw_testing = pd.read_csv('UNSW-testing-set.csv')
df_unsw_training.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.121478,tcp,-,FIN,6,4,258,172,74.087490,...,1,1,0,0,0,1,1,0,Normal,0
1,2,0.649902,tcp,-,FIN,14,38,734,42014,78.473372,...,1,2,0,0,0,1,6,0,Normal,0
2,3,1.623129,tcp,-,FIN,8,16,364,13186,14.170161,...,1,3,0,0,0,2,6,0,Normal,0
3,4,1.681642,tcp,ftp,FIN,12,12,628,770,13.677108,...,1,3,1,1,0,2,1,0,Normal,0
4,5,0.449454,tcp,-,FIN,10,6,534,268,33.373826,...,1,40,0,0,0,2,39,0,Normal,0


In [71]:
df_unsw_testing.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [72]:
# Check which column needs fixing
df_unsw_training.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175341 entries, 0 to 175340
Data columns (total 45 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id                 175341 non-null  int64  
 1   dur                175341 non-null  float64
 2   proto              175341 non-null  object 
 3   service            175341 non-null  object 
 4   state              175341 non-null  object 
 5   spkts              175341 non-null  int64  
 6   dpkts              175341 non-null  int64  
 7   sbytes             175341 non-null  int64  
 8   dbytes             175341 non-null  int64  
 9   rate               175341 non-null  float64
 10  sttl               175341 non-null  int64  
 11  dttl               175341 non-null  int64  
 12  sload              175341 non-null  float64
 13  dload              175341 non-null  float64
 14  sloss              175341 non-null  int64  
 15  dloss              175341 non-null  int64  
 16  si

In [73]:
# Check which column needs fixing
df_unsw_testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 45 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 82332 non-null  int64  
 1   dur                82332 non-null  float64
 2   proto              82332 non-null  object 
 3   service            82332 non-null  object 
 4   state              82332 non-null  object 
 5   spkts              82332 non-null  int64  
 6   dpkts              82332 non-null  int64  
 7   sbytes             82332 non-null  int64  
 8   dbytes             82332 non-null  int64  
 9   rate               82332 non-null  float64
 10  sttl               82332 non-null  int64  
 11  dttl               82332 non-null  int64  
 12  sload              82332 non-null  float64
 13  dload              82332 non-null  float64
 14  sloss              82332 non-null  int64  
 15  dloss              82332 non-null  int64  
 16  sinpkt             823

In [74]:
# Check which column needs fixing
for column in df_unsw_training.columns:
    unique_values = df_unsw_training[column].unique()
    print(f"Unique values in '{column}':")
    print(unique_values)
    print()
    

Unique values in 'id':
[     1      2      3 ... 175339 175340 175341]

Unique values in 'dur':
[0.121478 0.649902 1.623129 ... 3.71911  0.996503 1.557125]

Unique values in 'proto':
['tcp' 'udp' 'arp' 'ospf' 'icmp' 'igmp' 'rtp' 'ddp' 'ipv6-frag' 'cftp'
 'wsn' 'pvp' 'wb-expak' 'mtp' 'pri-enc' 'sat-mon' 'cphb' 'sun-nd' 'iso-ip'
 'xtp' 'il' 'unas' 'mfe-nsp' '3pc' 'ipv6-route' 'idrp' 'bna' 'swipe'
 'kryptolan' 'cpnx' 'rsvp' 'wb-mon' 'vmtp' 'ib' 'dgp' 'eigrp' 'ax.25'
 'gmtp' 'pnni' 'sep' 'pgm' 'idpr-cmtp' 'zero' 'rvd' 'mobile' 'narp' 'fc'
 'pipe' 'ipcomp' 'ipv6-no' 'sat-expak' 'ipv6-opts' 'snp' 'ipcv'
 'br-sat-mon' 'ttp' 'tcf' 'nsfnet-igp' 'sprite-rpc' 'aes-sp3-d' 'sccopmce'
 'sctp' 'qnx' 'scps' 'etherip' 'aris' 'pim' 'compaq-peer' 'vrrp' 'iatp'
 'stp' 'l2tp' 'srp' 'sm' 'isis' 'smp' 'fire' 'ptp' 'crtp' 'sps'
 'merit-inp' 'idpr' 'skip' 'any' 'larp' 'ipip' 'micp' 'encap' 'ifmp'
 'tp++' 'a/n' 'ipv6' 'i-nlsp' 'ipx-n-ip' 'sdrp' 'tlsp' 'gre' 'mhrp' 'ddx'
 'ippc' 'visa' 'secure-vmtp' 'uti' 'vines

In [75]:
# Check for null values
missing_values_unsw_training = df_unsw_training.isnull().sum()
print(missing_values_unsw_training)

# Check for duplicate rows
duplicate_unsw_training = df_unsw_training.duplicated().sum()
print(duplicate_unsw_training)

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

In [76]:
# Check for null values
missing_values_unsw_testing = df_unsw_testing.isnull().sum()
print(missing_values_unsw_testing)

# Check for duplicate rows
duplicate_unsw_testing = df_unsw_testing.duplicated().sum()
print(duplicate_unsw_testing)

id                   0
dur                  0
proto                0
service              0
state                0
spkts                0
dpkts                0
sbytes               0
dbytes               0
rate                 0
sttl                 0
dttl                 0
sload                0
dload                0
sloss                0
dloss                0
sinpkt               0
dinpkt               0
sjit                 0
djit                 0
swin                 0
stcpb                0
dtcpb                0
dwin                 0
tcprtt               0
synack               0
ackdat               0
smean                0
dmean                0
trans_depth          0
response_body_len    0
ct_srv_src           0
ct_state_ttl         0
ct_dst_ltm           0
ct_src_dport_ltm     0
ct_dst_sport_ltm     0
ct_dst_src_ltm       0
is_ftp_login         0
ct_ftp_cmd           0
ct_flw_http_mthd     0
ct_src_ltm           0
ct_srv_dst           0
is_sm_ips_ports      0
attack_cat 

## Preprocessing

### Remove Duplicates

#### KDD99

In [77]:
print(f"Number of rows before removing duplicates: {len(df_kdd99_testing)}")

# Remove duplicates and overwrite the original DataFrame
df_kdd99_testing = df_kdd99_testing.drop_duplicates()

# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_kdd99_testing)}")

# Check if any duplicates remain
remaining_duplicates_df_kdd99_testing = df_kdd99_testing.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates_df_kdd99_testing}")


Number of rows before removing duplicates: 311028
Number of rows after removing duplicates: 97325
Number of duplicate rows remaining: 0


#### UNSW

In [78]:
print(f"Number of rows before removing duplicates: {len(df_kdd99_training)}")

# Remove duplicates and overwrite the original DataFrame
df_kdd99_training = df_kdd99_training.drop_duplicates()

# Check number of rows after removing duplicates
print(f"Number of rows after removing duplicates: {len(df_kdd99_training)}")

# Check if any duplicates remain
remaining_duplicates = df_kdd99_training.duplicated().sum()
print(f"Number of duplicate rows remaining: {remaining_duplicates}")


Number of rows before removing duplicates: 494020
Number of rows after removing duplicates: 145585
Number of duplicate rows remaining: 0


### Label Encoding

#### KDD99

In [79]:
le = LabelEncoder() 
#training
df_kdd99_training['protocol_type'] = le.fit_transform(df_kdd99_training['protocol_type'])
df_kdd99_training['service'] = le.fit_transform(df_kdd99_training['service'])
df_kdd99_training['flag'] = le.fit_transform(df_kdd99_training['flag'])

#testing
df_kdd99_testing['protocol_type'] = le.fit_transform(df_kdd99_testing['protocol_type'])
df_kdd99_testing['service'] = le.fit_transform(df_kdd99_testing['service'])
df_kdd99_testing['flag'] = le.fit_transform(df_kdd99_testing['flag'])


#### UNSW

In [80]:
#training
df_unsw_training['state'] = le.fit_transform(df_unsw_training['state'])
df_unsw_training['service'] = le.fit_transform(df_unsw_training['service'])
df_unsw_training['proto'] = le.fit_transform(df_unsw_training['proto'])
#testing
df_unsw_testing['state'] = le.fit_transform(df_unsw_testing['state'])
df_unsw_testing['service'] = le.fit_transform(df_unsw_testing['service'])
df_unsw_testing['proto'] = le.fit_transform(df_unsw_testing['proto'])

### Remove Useless Columns

#### KDD99

In [81]:
columns_to_drop_kdd = ['label']

# training
df_autoencoder_kdd99_train = df_kdd99_training[df_kdd99_training['label'] == 'normal'].drop(columns=columns_to_drop_kdd)

y = df_kdd99_training['label']
df_kdd99_training = df_kdd99_training.drop(columns=columns_to_drop_kdd)
X = df_kdd99_training

df_kdd99_train, df_kdd99_validation, df_kdd99_train_label, df_kdd99_validation_label = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# testing
df_kdd99_testing_label = df_kdd99_testing['label']
df_kdd99_testing = df_kdd99_testing.drop(columns=columns_to_drop_kdd)

#### UNSW

In [82]:
columns_to_drop_unsw = ['attack_cat', 'id', 'label']

# training
df_autoencoder_unsw_train = df_unsw_training[df_unsw_training['label'] == 0].drop(columns=columns_to_drop_unsw)

y = df_unsw_training['label']
df_unsw_training = df_unsw_training.drop(columns=columns_to_drop_unsw)
X = df_unsw_training

df_unsw_train, df_unsw_validation, df_unsw_train_label, df_unsw_validation_label = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# testing
df_unsw_testing_label = df_unsw_testing['label']
df_unsw_testing = df_unsw_testing.drop(columns=columns_to_drop_unsw)

### Change labels to attack or not attack

In [83]:
df_kdd99_train_label = df_kdd99_train_label.apply(lambda x: 0 if x == 'normal' else 1)
df_kdd99_validation_label = df_kdd99_validation_label.apply(lambda x: 0 if x == 'normal' else 1)
df_kdd99_testing_label = df_kdd99_testing_label.apply(lambda x: 0 if x == 'normal.' else 1)

### Check distribution

In [84]:
# Training dataset anomaly percentage
anomalous_kdd99_train = (df_kdd99_train_label.sum() / len(df_kdd99_train_label)) * 100

# Validation dataset anomaly percentage
anomalous_kdd99_validation = (df_kdd99_validation_label.sum() / len(df_kdd99_validation_label)) * 100

# Testing dataset anomaly percentage
anomalous_kdd99_testing = (df_kdd99_testing_label.sum() / len(df_kdd99_testing_label)) * 100

print(f"KDD99 Training Anomaly Percentage: {anomalous_kdd99_train:.2f}%")
print(f"KDD99 Validation Anomaly Percentage: {anomalous_kdd99_validation:.2f}%")
print(f"KDD99 Testing Anomaly Percentage: {anomalous_kdd99_testing:.2f}%")


KDD99 Training Anomaly Percentage: 39.67%
KDD99 Validation Anomaly Percentage: 39.67%
KDD99 Testing Anomaly Percentage: 42.17%


In [85]:
# Training dataset anomaly percentage
anomalous_unsw_train = (df_unsw_train_label.sum() / len(df_unsw_train_label)) * 100

# Validation dataset anomaly percentage
anomalous_unsw_validation = (df_unsw_validation_label.sum() / len(df_unsw_validation_label)) * 100

# Testing dataset anomaly percentage
anomalous_unsw_testing = (df_unsw_testing_label.sum() / len(df_unsw_testing_label)) * 100

print(f"UNSW Training Anomaly Percentage: {anomalous_unsw_train:.2f}%")
print(f"UNSW Validation Anomaly Percentage: {anomalous_unsw_validation:.2f}%")
print(f"UNSW Testing Anomaly Percentage: {anomalous_unsw_testing:.2f}%")


UNSW Training Anomaly Percentage: 68.06%
UNSW Validation Anomaly Percentage: 68.06%
UNSW Testing Anomaly Percentage: 55.06%


### Normalize Data

In [86]:
scaler = RobustScaler()

# kdd
df_kdd99_testing = pd.DataFrame(scaler.fit_transform(df_kdd99_testing), columns=df_kdd99_testing.columns)
df_autoencoder_kdd99_train = pd.DataFrame(scaler.fit_transform(df_autoencoder_kdd99_train),columns=df_autoencoder_kdd99_train.columns)
df_kdd99_train = pd.DataFrame(scaler.transform(df_kdd99_train),columns=df_kdd99_train.columns)
df_kdd99_validation = pd.DataFrame(scaler.transform(df_kdd99_validation),columns=df_kdd99_validation.columns)

# unsw
df_unsw_testing = pd.DataFrame(scaler.fit_transform(df_unsw_testing), columns=df_unsw_testing.columns)
df_autoencoder_unsw_train = pd.DataFrame(scaler.fit_transform(df_autoencoder_unsw_train),columns=df_autoencoder_unsw_train.columns)
df_unsw_train= pd.DataFrame(scaler.transform(df_unsw_train),columns=df_unsw_train.columns)
df_unsw_validation = pd.DataFrame(scaler.transform(df_unsw_validation),columns=df_unsw_validation.columns)


### Final Check for Dataset

#### KDD99

In [87]:
# kdd
df_kdd99_train_label.unique()


array([0, 1], dtype=int64)

In [88]:
df_kdd99_testing_label.unique()


array([1, 0], dtype=int64)

In [89]:
df_kdd99_testing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97325 entries, 0 to 97324
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     97325 non-null  float64
 1   protocol_type                97325 non-null  float64
 2   service                      97325 non-null  float64
 3   flag                         97325 non-null  float64
 4   src_bytes                    97325 non-null  float64
 5   dst_bytes                    97325 non-null  float64
 6   land                         97325 non-null  float64
 7   wrong_fragment               97325 non-null  float64
 8   urgent                       97325 non-null  float64
 9   hot                          97325 non-null  float64
 10  num_failed_logins            97325 non-null  float64
 11  logged_in                    97325 non-null  float64
 12  num_compromised              97325 non-null  float64
 13  root_shell      

In [90]:
df_autoencoder_kdd99_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87831 entries, 0 to 87830
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     87831 non-null  float64
 1   protocol_type                87831 non-null  float64
 2   service                      87831 non-null  float64
 3   flag                         87831 non-null  float64
 4   src_bytes                    87831 non-null  float64
 5   dst_bytes                    87831 non-null  float64
 6   land                         87831 non-null  float64
 7   wrong_fragment               87831 non-null  float64
 8   urgent                       87831 non-null  float64
 9   hot                          87831 non-null  float64
 10  num_failed_logins            87831 non-null  float64
 11  logged_in                    87831 non-null  float64
 12  num_compromised              87831 non-null  float64
 13  root_shell      

In [91]:
df_kdd99_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116468 entries, 0 to 116467
Data columns (total 41 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     116468 non-null  float64
 1   protocol_type                116468 non-null  float64
 2   service                      116468 non-null  float64
 3   flag                         116468 non-null  float64
 4   src_bytes                    116468 non-null  float64
 5   dst_bytes                    116468 non-null  float64
 6   land                         116468 non-null  float64
 7   wrong_fragment               116468 non-null  float64
 8   urgent                       116468 non-null  float64
 9   hot                          116468 non-null  float64
 10  num_failed_logins            116468 non-null  float64
 11  logged_in                    116468 non-null  float64
 12  num_compromised              116468 non-null  float64
 13 

In [92]:
df_kdd99_validation.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29117 entries, 0 to 29116
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     29117 non-null  float64
 1   protocol_type                29117 non-null  float64
 2   service                      29117 non-null  float64
 3   flag                         29117 non-null  float64
 4   src_bytes                    29117 non-null  float64
 5   dst_bytes                    29117 non-null  float64
 6   land                         29117 non-null  float64
 7   wrong_fragment               29117 non-null  float64
 8   urgent                       29117 non-null  float64
 9   hot                          29117 non-null  float64
 10  num_failed_logins            29117 non-null  float64
 11  logged_in                    29117 non-null  float64
 12  num_compromised              29117 non-null  float64
 13  root_shell      

#### UNSW

In [93]:
df_unsw_testing.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82332 entries, 0 to 82331
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                82332 non-null  float64
 1   proto              82332 non-null  float64
 2   service            82332 non-null  float64
 3   state              82332 non-null  float64
 4   spkts              82332 non-null  float64
 5   dpkts              82332 non-null  float64
 6   sbytes             82332 non-null  float64
 7   dbytes             82332 non-null  float64
 8   rate               82332 non-null  float64
 9   sttl               82332 non-null  float64
 10  dttl               82332 non-null  float64
 11  sload              82332 non-null  float64
 12  dload              82332 non-null  float64
 13  sloss              82332 non-null  float64
 14  dloss              82332 non-null  float64
 15  sinpkt             82332 non-null  float64
 16  dinpkt             823

In [94]:
df_autoencoder_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                56000 non-null  float64
 1   proto              56000 non-null  float64
 2   service            56000 non-null  float64
 3   state              56000 non-null  float64
 4   spkts              56000 non-null  float64
 5   dpkts              56000 non-null  float64
 6   sbytes             56000 non-null  float64
 7   dbytes             56000 non-null  float64
 8   rate               56000 non-null  float64
 9   sttl               56000 non-null  float64
 10  dttl               56000 non-null  float64
 11  sload              56000 non-null  float64
 12  dload              56000 non-null  float64
 13  sloss              56000 non-null  float64
 14  dloss              56000 non-null  float64
 15  sinpkt             56000 non-null  float64
 16  dinpkt             560

In [95]:
df_unsw_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140272 entries, 0 to 140271
Data columns (total 42 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   dur                140272 non-null  float64
 1   proto              140272 non-null  float64
 2   service            140272 non-null  float64
 3   state              140272 non-null  float64
 4   spkts              140272 non-null  float64
 5   dpkts              140272 non-null  float64
 6   sbytes             140272 non-null  float64
 7   dbytes             140272 non-null  float64
 8   rate               140272 non-null  float64
 9   sttl               140272 non-null  float64
 10  dttl               140272 non-null  float64
 11  sload              140272 non-null  float64
 12  dload              140272 non-null  float64
 13  sloss              140272 non-null  float64
 14  dloss              140272 non-null  float64
 15  sinpkt             140272 non-null  float64
 16  di

In [96]:
df_unsw_validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35069 entries, 0 to 35068
Data columns (total 42 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dur                35069 non-null  float64
 1   proto              35069 non-null  float64
 2   service            35069 non-null  float64
 3   state              35069 non-null  float64
 4   spkts              35069 non-null  float64
 5   dpkts              35069 non-null  float64
 6   sbytes             35069 non-null  float64
 7   dbytes             35069 non-null  float64
 8   rate               35069 non-null  float64
 9   sttl               35069 non-null  float64
 10  dttl               35069 non-null  float64
 11  sload              35069 non-null  float64
 12  dload              35069 non-null  float64
 13  sloss              35069 non-null  float64
 14  dloss              35069 non-null  float64
 15  sinpkt             35069 non-null  float64
 16  dinpkt             350

In [97]:
df_unsw_train_label.unique()

array([1, 0], dtype=int64)

In [98]:
df_kdd99_testing_label.unique()

array([1, 0], dtype=int64)

## **Model Generation without Optimizer + Evaluation**

### **DBSCAN**

#### ----KDD99

In [99]:
# 1. Train DBSCAN on training data
dbscan_default_kdd99 = DBSCAN(eps=0.5, min_samples=5)
dbscan_default_kdd99.fit(df_kdd99_training)

# 2. Identify core points from training data (these represent "normal" clusters)
core_samples_mask = np.zeros_like(dbscan_default_kdd99.labels_, dtype=bool)
core_samples_mask[dbscan_default_kdd99.core_sample_indices_] = True
core_points = df_kdd99_training[core_samples_mask]

# 3. For each test point, calculate distance to nearest core point
nbrs = NearestNeighbors(n_neighbors=1).fit(core_points)
distances, _ = nbrs.kneighbors(df_kdd99_testing)

# 4. Set threshold (e.g., 95th percentile of training point distances)
# First calculate distances of training  points to their own core points
train_distances, _ = nbrs.kneighbors(df_kdd99_training)
threshold = np.percentile(train_distances, 60)  # Adjust this percentile to tune recall

# 5. Classify test points: 0=normal (close to core), 1=anomaly (far from core)
dbscan_default_kdd99_pred_labels = (distances > threshold).astype(int).flatten()

# Evaluation
dbscan_default_kdd99_accuracy = accuracy_score(df_kdd99_testing_label, dbscan_default_kdd99_pred_labels)
dbscan_default_kdd99_precision = precision_score(df_kdd99_testing_label, dbscan_default_kdd99_pred_labels)
dbscan_default_kdd99_recall = recall_score(df_kdd99_testing_label, dbscan_default_kdd99_pred_labels)
dbscan_default_kdd99_f1 = f1_score(df_kdd99_testing_label, dbscan_default_kdd99_pred_labels)
dbscan_default_kdd99_auc = roc_auc_score(df_kdd99_testing_label, dbscan_default_kdd99_pred_labels)
dbscan_default_kdd99_conf_matrix = confusion_matrix(df_kdd99_testing_label, dbscan_default_kdd99_pred_labels)

# Print the evaluation metrics
print(f"DBSCAN Distance-Based KDD99 Accuracy: {dbscan_default_kdd99_accuracy:.4f}")
print(f"DBSCAN Distance-Based KDD99 Precision: {dbscan_default_kdd99_precision:.4f}")
print(f"DBSCAN Distance-Based KDD99 Recall: {dbscan_default_kdd99_recall:.4f}")
print(f"DBSCAN Distance-Based KDD99 F1 Score: {dbscan_default_kdd99_f1:.4f}")
print(f"DBSCAN Distance-Based KDD99 AUC: {dbscan_default_kdd99_auc:.4f}")
print("DBSCAN Distance-Based KDD99 Confusion Matrix:")
print(dbscan_default_kdd99_conf_matrix)

DBSCAN Distance-Based KDD99 Accuracy: 0.5651
DBSCAN Distance-Based KDD99 Precision: 0.0537
DBSCAN Distance-Based KDD99 Recall: 0.0019
DBSCAN Distance-Based KDD99 F1 Score: 0.0036
DBSCAN Distance-Based KDD99 AUC: 0.4889
DBSCAN Distance-Based KDD99 Confusion Matrix:
[[54923  1356]
 [40969    77]]


#### ----UNSW

In [58]:
# 1. Train DBSCAN on training data
dbscan_default_unsw = DBSCAN(eps=0.5, min_samples=5)
dbscan_default_unsw.fit(df_unsw_training)

# 2. Identify core points from training data (these represent "normal" clusters)
core_samples_mask = np.zeros_like(dbscan_default_unsw.labels_, dtype=bool)
core_samples_mask[dbscan_default_unsw.core_sample_indices_] = True
core_points = df_unsw_training[core_samples_mask]

# 3. For each test point, calculate distance to nearest core point
nbrs = NearestNeighbors(n_neighbors=1).fit(core_points)
distances, _ = nbrs.kneighbors(df_unsw_testing)

# 4. Set threshold (e.g., 40th percentile of training point distances)
# First calculate distances of training points to their own core points
train_distances, _ = nbrs.kneighbors(df_unsw_training)
threshold = np.percentile(train_distances, 40)  # Adjust this percentile to tune recall

# 5. Classify test points: 0=normal (close to core), 1=anomaly (far from core)
dbscan_default_unsw_pred_labels = (distances > threshold).astype(int).flatten()

# Evaluation
dbscan_default_unsw_accuracy = accuracy_score(df_unsw_testing_label, dbscan_default_unsw_pred_labels)
dbscan_default_unsw_precision = precision_score(df_unsw_testing_label, dbscan_default_unsw_pred_labels)
dbscan_default_unsw_recall = recall_score(df_unsw_testing_label, dbscan_default_unsw_pred_labels)
dbscan_default_unsw_f1 = f1_score(df_unsw_testing_label, dbscan_default_unsw_pred_labels)
dbscan_default_unsw_auc = roc_auc_score(df_unsw_testing_label, dbscan_default_unsw_pred_labels)
dbscan_default_unsw_conf_matrix = confusion_matrix(df_unsw_testing_label, dbscan_default_unsw_pred_labels)

# Print the evaluation metrics
print(f"DBSCAN Distance-Based UNSW Accuracy: {dbscan_default_unsw_accuracy:.4f}")
print(f"DBSCAN Distance-Based UNSW Precision: {dbscan_default_unsw_precision:.4f}")
print(f"DBSCAN Distance-Based UNSW Recall: {dbscan_default_unsw_recall:.4f}")
print(f"DBSCAN Distance-Based UNSW F1 Score: {dbscan_default_unsw_f1:.4f}")
print(f"DBSCAN Distance-Based UNSW AUC: {dbscan_default_unsw_auc:.4f}")
print("DBSCAN Distance-Based UNSW Confusion Matrix:")
print(dbscan_default_unsw_conf_matrix)

DBSCAN Distance-Based UNSW Accuracy: 0.5234
DBSCAN Distance-Based UNSW Precision: 0.5389
DBSCAN Distance-Based UNSW Recall: 0.9304
DBSCAN Distance-Based UNSW F1 Score: 0.6825
DBSCAN Distance-Based UNSW AUC: 0.4775
DBSCAN Distance-Based UNSW Confusion Matrix:
[[  911 36089]
 [ 3153 42179]]


### **Autoencoder**

#### ----Architecture

In [47]:
# Define the Autoencoder architecture
def build_autoencoder(input_dim, latent_dim=32, hidden_layers=[64, 32], activation='relu', 
                      output_activation='sigmoid', dropout_rate=0.2):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    x = Dense(hidden_layers[0], activation=activation)(input_layer)
    x = Dropout(dropout_rate)(x)
    x = Dense(hidden_layers[1], activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    
    # Latent space
    latent_space = Dense(latent_dim, activation=activation)(x)
    
    # Decoder
    x = Dense(hidden_layers[1], activation=activation)(latent_space)
    x = Dropout(dropout_rate)(x)
    x = Dense(hidden_layers[0], activation=activation)(x)
    x = Dropout(dropout_rate)(x)
    
    # Output layer
    output_layer = Dense(input_dim, activation=output_activation)(x)
    
    # Build the model
    autoencoder = Model(input_layer, output_layer)
    return autoencoder

# Default parameters
input_dim_kdd99 = 41
input_dim_unsw = 42

loss_function = 'mse'
optimizer = Adam(learning_rate=0.001)
batch_size = 32
epochs = 50

# Build the Autoencoder for KDD99
autoencoder_kdd99 = build_autoencoder(input_dim=input_dim_kdd99)

# Compile the model
autoencoder_kdd99.compile(optimizer=optimizer, loss=loss_function)

# Print the model summary
print("Autoencoder for KDD99:")
autoencoder_kdd99.summary()

# Build the Autoencoder for UNSW-NB15
autoencoder_unsw = build_autoencoder(input_dim=input_dim_unsw)

# Compile the model
autoencoder_unsw.compile(optimizer=optimizer, loss=loss_function)

# Print the model summary
print("\nAutoencoder for UNSW-NB15:")
autoencoder_unsw.summary()



Autoencoder for KDD99:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 41)]              0         
                                                                 
 dense (Dense)               (None, 64)                2688      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dense_3 (Dense)             (None, 3

#### ----KDD99

In [ ]:
print("\nTraining Autoencoder for KDD99...")
history_autoencoder_kdd99 = autoencoder_kdd99.fit(
    df_autoencoder_kdd99_train,  
    df_autoencoder_kdd99_train,  
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(df_kdd99_testing, df_kdd99_testing),  
    verbose=1
)

# Function to calculate reconstruction error
def calculate_reconstruction_error(model, data):
    reconstructed_data = model.predict(data)
    reconstruction_error = np.mean(np.square(data - reconstructed_data), axis=1)
    return reconstruction_error

# Calculate reconstruction error for KDD99
reconstruction_error_autoencoder_kdd99_train = calculate_reconstruction_error(autoencoder_kdd99, df_autoencoder_kdd99_train)
reconstruction_error_autoencoder_kdd99_test = calculate_reconstruction_error(autoencoder_kdd99, df_kdd99_testing)

# Define a threshold for anomaly detection
threshold_autoencoder_kdd99 = np.percentile(reconstruction_error_autoencoder_kdd99_train, 60)

# Classify samples as normal (0) or anomalous (1) based on the threshold
def classify_anomalies(reconstruction_error, threshold):
    return np.where(reconstruction_error > threshold, 1, 0)

# Predict anomalies for KDD99
y_pred_autoencoder_kdd99 = classify_anomalies(reconstruction_error_autoencoder_kdd99_test, threshold_autoencoder_kdd99)

# Evaluation metrics for KDD99
accuracy_autoencoder_kdd99 = accuracy_score(df_kdd99_testing_label, y_pred_autoencoder_kdd99)
precision_autoencoder_kdd99 = precision_score(df_kdd99_testing_label, y_pred_autoencoder_kdd99)
recall_autoencoder_kdd99 = recall_score(df_kdd99_testing_label, y_pred_autoencoder_kdd99)
f1_autoencoder_kdd99 = f1_score(df_kdd99_testing_label, y_pred_autoencoder_kdd99)
auc_autoencoder_kdd99 = roc_auc_score(df_kdd99_testing_label, reconstruction_error_autoencoder_kdd99_test)
conf_matrix_autoencoder_kdd99 = confusion_matrix(df_kdd99_testing_label, y_pred_autoencoder_kdd99)

# Print evaluation metrics for KDD99
print("\nEvaluation Metrics for Autoencoder - KDD99:")
print(f"Accuracy: {accuracy_autoencoder_kdd99:.4f}")
print(f"Precision: {precision_autoencoder_kdd99:.4f}")
print(f"Recall: {recall_autoencoder_kdd99:.4f}")
print(f"F1 Score: {f1_autoencoder_kdd99:.4f}")
print(f"AUC: {auc_autoencoder_kdd99:.4f}")
print("Confusion Matrix:")
print(conf_matrix_autoencoder_kdd99)


Training Autoencoder for KDD99...
Epoch 1/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.6953 - val_loss: 81658.0859
Epoch 2/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.6445 - val_loss: 81658.0859
Epoch 3/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.7461 - val_loss: 81658.0859
Epoch 4/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.7656 - val_loss: 81658.0859
Epoch 5/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.6953 - val_loss: 81658.1250
Epoch 6/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.7109 - val_loss: 81658.0859
Epoch 7/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.6836 - val_loss: 81658.0859
Epoch 8/50
2745/2745 [==============================] - 11s 4ms/step - loss: 45411.6406 - val_loss: 81658.0859
Epoch 9/50
2745/2745 [==============================] - 11s 4ms/step - loss: 

#### ----UNSW

In [60]:
print("\nTraining Autoencoder for UNSW-NB15...")
history_autoencoder_unsw = autoencoder_unsw.fit(
    df_autoencoder_unsw_train,  
    df_autoencoder_unsw_train,  
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(df_unsw_testing, df_unsw_testing),  
    verbose=1
)

# Calculate reconstruction error for UNSW-NB15
reconstruction_error_autoencoder_unsw_train = calculate_reconstruction_error(autoencoder_unsw, df_autoencoder_unsw_train)
reconstruction_error_autoencoder_unsw_test = calculate_reconstruction_error(autoencoder_unsw, df_unsw_testing)

# Define a threshold for anomaly detection
threshold_autoencoder_unsw = np.percentile(reconstruction_error_autoencoder_unsw_train, 40)

# Predict anomalies for UNSW-NB15
y_pred_autoencoder_unsw = classify_anomalies(reconstruction_error_autoencoder_unsw_test, threshold_autoencoder_unsw)

# Evaluation metrics for UNSW-NB15
accuracy_autoencoder_unsw = accuracy_score(df_unsw_testing_label, y_pred_autoencoder_unsw)
precision_autoencoder_unsw = precision_score(df_unsw_testing_label, y_pred_autoencoder_unsw)
recall_autoencoder_unsw = recall_score(df_unsw_testing_label, y_pred_autoencoder_unsw)
f1_autoencoder_unsw = f1_score(df_unsw_testing_label, y_pred_autoencoder_unsw)
auc_autoencoder_unsw = roc_auc_score(df_unsw_testing_label, reconstruction_error_autoencoder_unsw_test)
conf_matrix_autoencoder_unsw = confusion_matrix(df_unsw_testing_label, y_pred_autoencoder_unsw)

# Print evaluation metrics for UNSW-NB15
print("\nEvaluation Metrics for Autoencoder - UNSW-NB15:")
print(f"Accuracy: {accuracy_autoencoder_unsw:.4f}")
print(f"Precision: {precision_autoencoder_unsw:.4f}")
print(f"Recall: {recall_autoencoder_unsw:.4f}")
print(f"F1 Score: {f1_autoencoder_unsw:.4f}")
print(f"AUC: {auc_autoencoder_unsw:.4f}")
print("Confusion Matrix:")
print(conf_matrix_autoencoder_unsw)


Training Autoencoder for UNSW-NB15...
Epoch 1/50
1750/1750 [==============================] - 8s 5ms/step - loss: 45643892.0000 - val_loss: 34564204.0000
Epoch 2/50
1750/1750 [==============================] - 8s 4ms/step - loss: 45643828.0000 - val_loss: 34564204.0000
Epoch 3/50
1750/1750 [==============================] - 8s 4ms/step - loss: 45643852.0000 - val_loss: 34564204.0000
Epoch 4/50
1750/1750 [==============================] - 8s 5ms/step - loss: 45643840.0000 - val_loss: 34564204.0000
Epoch 5/50
1750/1750 [==============================] - 8s 4ms/step - loss: 45643852.0000 - val_loss: 34564204.0000
Epoch 6/50
1750/1750 [==============================] - 8s 4ms/step - loss: 45643844.0000 - val_loss: 34564204.0000
Epoch 7/50
1750/1750 [==============================] - 8s 4ms/step - loss: 45643880.0000 - val_loss: 34564204.0000
Epoch 8/50
1750/1750 [==============================] - 8s 4ms/step - loss: 45643844.0000 - val_loss: 34564204.0000
Epoch 9/50
1750/1750 [===========

### **Isolation Forest**

#### ----Architecture

In [42]:
IF_n_estimators = 100
IF_max_samples = 'auto'  # If data < 256, use all; otherwise, use 256
IF_contamination = 'auto'  # Automatically infer contamination rate
IF_max_features = 1.0  # Use all features
IF_bootstrap = False
IF_random_state = None

#### ----KDD99

In [44]:
IF_kdd99 = IsolationForest(
    n_estimators=IF_n_estimators,
    max_samples=IF_max_samples,
    contamination=IF_contamination,
    max_features=IF_max_features,
    bootstrap=IF_bootstrap,
    random_state=IF_random_state
)

# Train the Isolation Forest on KDD99 training data
IF_kdd99.fit(df_kdd99_training)

# Get anomaly scores (lower values = more anomalous)
IF_kdd99_scores = -IF_kdd99.decision_function(df_kdd99_testing)  # Convert to positive (higher = more anomalous)
contamination_rate = np.mean(IF_kdd99.predict(df_kdd99_training) == -1)

# Calculate threshold based on contamination
# threshold = np.percentile(IF_kdd99_scores, 100 * (1 - contamination_rate))
threshold = np.percentile(IF_kdd99_scores, 60)

# Convert scores to binary predictions (0=normal, 1=anomaly)
IF_kdd99_predictions_binary = (IF_kdd99_scores > threshold).astype(int)

# Evaluate the model for KDD99
IF_kdd99_accuracy = accuracy_score(df_kdd99_testing_label, IF_kdd99_predictions_binary)
IF_kdd99_precision = precision_score(df_kdd99_testing_label, IF_kdd99_predictions_binary)
IF_kdd99_recall = recall_score(df_kdd99_testing_label, IF_kdd99_predictions_binary)
IF_kdd99_f1 = f1_score(df_kdd99_testing_label, IF_kdd99_predictions_binary)
IF_kdd99_auc = roc_auc_score(df_kdd99_testing_label, IF_kdd99_predictions_binary)
IF_kdd99_conf_matrix = confusion_matrix(df_kdd99_testing_label, IF_kdd99_predictions_binary)

# Print evaluation metrics for KDD99
print("Evaluation Metrics for Isolation Forest (KDD99):")
print(f"Accuracy: {IF_kdd99_accuracy:.4f}")
print(f"Precision: {IF_kdd99_precision:.4f}")
print(f"Recall: {IF_kdd99_recall:.4f}")
print(f"F1 Score: {IF_kdd99_f1:.4f}")
print(f"AUC: {IF_kdd99_auc:.4f}")
print("Confusion Matrix:")
print(IF_kdd99_conf_matrix)

Evaluation Metrics for Isolation Forest (KDD99):
Accuracy: 0.8717
Precision: 0.8718
Recall: 0.8157
F1 Score: 0.8428
AUC: 0.8641
Confusion Matrix:
[[51354  4925]
 [ 7564 33482]]


#### ----UNSW

In [46]:
# Initialize Isolation Forest for UNSW-NB15
IF_unsw = IsolationForest(
    n_estimators=IF_n_estimators,
    max_samples=IF_max_samples,
    contamination=IF_contamination,
    max_features=IF_max_features,
    bootstrap=IF_bootstrap,
    random_state=IF_random_state
)


# Train the Isolation Forest on UNSW-NB15 training data
IF_unsw.fit(df_unsw_training)

IF_unsw_scores = -IF_unsw.decision_function(df_unsw_testing)  # Convert to positive (higher = more anomalous)
contamination_rate = np.mean(IF_unsw.predict(df_unsw_training) == -1)
# Calculate threshold based on contamination
print(contamination_rate)
threshold = np.percentile(IF_unsw_scores, 40)

# Convert scores to binary predictions (0=normal, 1=anomaly)
IF_unsw_predictions_binary = (IF_unsw_scores > threshold).astype(int)

# Evaluate the model for UNSW-NB15
IF_unsw_accuracy = accuracy_score(df_unsw_testing_label, IF_unsw_predictions_binary)
IF_unsw_precision = precision_score(df_unsw_testing_label, IF_unsw_predictions_binary)
IF_unsw_recall = recall_score(df_unsw_testing_label, IF_unsw_predictions_binary)
IF_unsw_f1 = f1_score(df_unsw_testing_label, IF_unsw_predictions_binary)
IF_unsw_auc = roc_auc_score(df_unsw_testing_label, IF_unsw_predictions_binary)
IF_unsw_conf_matrix = confusion_matrix(df_unsw_testing_label, IF_unsw_predictions_binary)

# Print evaluation metrics for UNSW-NB15
print("\nEvaluation Metrics for Isolation Forest (UNSW-NB15):")
print(f"Accuracy: {IF_unsw_accuracy:.4f}")
print(f"Precision: {IF_unsw_precision:.4f}")
print(f"Recall: {IF_unsw_recall:.4f}")
print(f"F1 Score: {IF_unsw_f1:.4f}")
print(f"AUC: {IF_unsw_auc:.4f}")
print("Confusion Matrix:")
print(IF_unsw_conf_matrix)

0.035582094319069696

Evaluation Metrics for Isolation Forest (UNSW-NB15):
Accuracy: 0.4507
Precision: 0.5011
Recall: 0.5443
F1 Score: 0.5218
AUC: 0.4402
Confusion Matrix:
[[12435 24565]
 [20656 24676]]


## **Model Generation with Optimizer + Evaluation**

### +++++++++++++++++++++++++++++++++++++++++++++++ **Autoencoder** +++++++++++++++++++++++++++++++++++++++++++++++

#### ************Architecture************

In [ ]:
def create_autoencoder(
    input_dim,latent_dim,neurons_per_layer,activation_function, output_activation, loss_function, optimizer, learning_rate, dropout_rate):
    # Input Layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    x = Dense(neurons_per_layer, activation=activation_function)(input_layer)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons_per_layer, activation=activation_function)(x)  # 2nd hidden layer
    x = Dropout(dropout_rate)(x)
    encoded = Dense(latent_dim, activation=activation_function)(x)  # Latent space
    
    # Decoder
    x = Dense(neurons_per_layer, activation=activation_function)(encoded)
    x = Dropout(dropout_rate)(x)
    x = Dense(neurons_per_layer, activation=activation_function)(x)  # 2nd hidden layer
    x = Dropout(dropout_rate)(x)
    decoded = Dense(input_dim, activation=output_activation)(x)  # Output layer
    
    # Autoencoder Model
    autoencoder = Model(input_layer, decoded)

    # Optimizer Selection
    if optimizer == "Adam":
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == "SGD":
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == "Nadam":
        opt = Nadam(learning_rate=learning_rate)
    
    # Compile Model
    autoencoder.compile(optimizer=opt, loss=loss_function)
    
    return autoencoder

In [ ]:
def objective_function(solution, X_train, X_val, input_dim):
    (latent_dim, neurons_per_layer, learning_rate, dropout_rate, batch_size, epochs,
     activation_idx, output_activation_idx, loss_idx, optimizer_idx,) = solution

    # Map indices to actual values
    activation_functions = ["relu", "tanh", "sigmoid"]
    output_activations = ["sigmoid", "tanh", "linear", "softmax", "relu"]
    loss_functions = ["mean_squared_error", "mean_absolute_error",
                      "binary_crossentropy", "categorical_crossentropy"]
    optimizers = ["Adam", "SGD", "Nadam"]

    activation_function = activation_functions[int(activation_idx)]
    output_activation = output_activations[int(output_activation_idx)]
    loss_function = loss_functions[int(loss_idx)]
    optimizer = optimizers[int(optimizer_idx)]

    # Create Autoencoder
    autoencoder = create_autoencoder(
        input_dim=input_dim,
        latent_dim=int(latent_dim),
        neurons_per_layer=int(neurons_per_layer),
        activation_function=activation_function,
        output_activation=output_activation,
        loss_function=loss_function,
        optimizer=optimizer,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
    )

    # Train Autoencoder
    autoencoder.fit(
        X_train,
        X_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        verbose=0,
    )

    # Compute MSE (fitness value)
    reconstructed_val = autoencoder.predict(X_val)
    mse_val = np.mean(np.square(X_val - reconstructed_val))

    return mse_val


In [ ]:
def optimize_autoencoder(X_train, X_val, input_dim, optimizer_type, epochs=50, pop_size=20):
    # Define the search space
    problem_dict = {
        "bounds": FloatVar(
            lb=[
                16,    # latent_dim (16-64)
                16,    # neurons_per_layer (16-128)
                0.001, # learning_rate (0.001-0.01)
                0.1,   # dropout_rate (0.1-0.5)
                16,    # batch_size (16-64)
                20,    # epochs (20-100)
                0,     # activation_function index (0: relu, 1: tanh, 2: sigmoid)
                0,     # output_activation index (0: sigmoid, 1: tanh, 2: linear, 3: softmax, 4: relu)
                0,     # loss_function index (0: mse, 1: mae, 2: binary_crossentropy, 3: categorical_crossentropy)
                0,     # optimizer index (0: Adam, 1: SGD, 2: Nadam)
            ],
            ub=[
                64,128,0.01,0.5,64,100,2,4,3,2,],
            name="delta",
        ),
        "minmax": "min",  # Minimize MSE
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, input_dim),
    }
    
    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=10)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size, miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1)
    else:
        raise ValueError("Optimizer not supported. Choose 'WOA' or 'DE'.")
    
    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution

In [ ]:
def evaluate_model(autoencoder, X_test, y_test):
    # Reconstruct test data
    reconstructed = autoencoder.predict(X_test)
    mse = np.mean(np.square(X_test - reconstructed), axis=1)
    
    # Convert MSE to binary predictions (anomaly if MSE > 95th percentile)
    threshold = np.percentile(mse, 95)
    y_pred = (mse > threshold).astype(int)
    
    # Compute Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, mse)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return accuracy, precision, recall, f1, auc, conf_matrix

In [ ]:
def main_pipeline(dataset_name, optimizer_type):
    # Load Dataset
    if dataset_name == "kdd99":
        X_train = df_kdd99_train
        X_val = df_kdd99_validation
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
    elif dataset_name == "unsw":
        X_train = df_unsw_train
        X_val = df_unsw_validation
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
    else:
        raise ValueError("Dataset not supported. Choose 'kdd99' or 'unsw'.")

    input_dim = X_train.shape[1]

    # Optimize Autoencoder
    best_params = optimize_autoencoder(X_train, X_val, input_dim, optimizer_type)

    # Map indices to actual values
    activation_functions = ["relu", "tanh", "sigmoid"]
    output_activations = ["sigmoid", "tanh", "linear", "softmax", "relu"]
    loss_functions = ["mean_squared_error", "mean_absolute_error",
                      "binary_crossentropy", "categorical_crossentropy"]
    optimizers = ["Adam", "SGD", "Nadam"]

    (
        latent_dim, neurons_per_layer, learning_rate, dropout_rate, batch_size, epochs, activation_idx, output_activation_idx,
        loss_idx, optimizer_idx,) = best_params

    activation_function = activation_functions[int(activation_idx)]
    output_activation = output_activations[int(output_activation_idx)]
    loss_function = loss_functions[int(loss_idx)]
    optimizer = optimizers[int(optimizer_idx)]

    # Train Final Model
    autoencoder = create_autoencoder(
        input_dim=input_dim,
        latent_dim=int(latent_dim),
        neurons_per_layer=int(neurons_per_layer),
        activation_function=activation_function,
        output_activation=output_activation,
        loss_function=loss_function,
        optimizer=optimizer,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
        batch_size=int(batch_size),
        epochs=int(epochs),
    )

    autoencoder.fit(
        X_train,
        X_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        verbose=0,
    )

    # Evaluate
    accuracy, precision, recall, f1, auc, conf_matrix = evaluate_model(
        autoencoder, X_test, y_test)
    
    print(f"\n=== For Dataset {dataset_name}, {optimizer_type} optimizer===")


    print("\n=== Best Parameters ===")
    print(f"Latent Dim: {int(latent_dim)}")
    print(f"Neurons/Layer: {int(neurons_per_layer)}")
    print(f"Learning Rate: {learning_rate}")
    print(f"Dropout Rate: {dropout_rate}")
    print(f"Batch Size: {int(batch_size)}")
    print(f"Epochs: {int(epochs)}")
    print(f"Activation: {activation_function}")
    print(f"Output Activation: {output_activation}")
    print(f"Loss Function: {loss_function}")
    print(f"Optimizer: {optimizer}")

    print("\n=== Evaluation Metrics ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")

#### ************Autoencoder + Whale Optimization Algorithm************

In [ ]:
main_pipeline('kdd99', 'WOA')

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 04:04:28 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 756us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 855us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/

2025/03/27 04:09:08 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: -0.8090659340659341, Global best: -0.8090659340659341, Runtime: 162.73575 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 788us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 865us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 825us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 880us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 796us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 823us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 05:08:25 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 19, Current best: -0.8777472527472527, Global best: -0.9217032967032966, Runtime: 200.24437 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 808us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 816us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 826us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 05:11:35 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 20, Current best: -0.9381868131868132, Global best: -0.9381868131868132, Runtime: 189.48011 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 794us/step
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 721us/step

=== For Dataset kdd99, WOA optimizer ===
Threshold Percentile Used: 60
Calculated Threshold Value: 93.7520

=== Best Parameters ===
Latent Dim: 48
Neurons/Layer: 128
Learning Rate: 0.007759272157046315
Dropout Rate: 0.5
Batch Size: 256
Epochs: 100
Activation: sigmoid
Output Activation: relu
Loss Function: categorical_crossentropy
Optimizer: Nadam

=== Evaluation Metrics ===
Accuracy: 0.5059
Precision: 0.4187
Recall: 0.4603
F1 Score: 0.4385
AUC: 0.5253
Confusion Matrix:
[[3045 2607]
 [2202 1878]]


In [ ]:
main_pipeline('unsw', 'WOA')

2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 817us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step


2025/03/27 03:32:13 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.


2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 702us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 983us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 762us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 830us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 697us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 798us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 815us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 707us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 739us/step


2025/03/27 03:37:22 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: -0.7116788321167883, Global best: -0.7116788321167883, Runtime: 142.31599 seconds


2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 786us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step


...

2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 881us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 729us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 866us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 717us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 797us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step


2025/03/27 04:34:47 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 19, Current best: -0.7212591240875912, Global best: -0.7214872262773723, Runtime: 289.56987 seconds


2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 864us/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step
2603/2603 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
137/137 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step


2025/03/27 04:38:04 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 20, Current best: -0.7212591240875912, Global best: -0.7214872262773723, Runtime: 197.28513 seconds


2603/2603 ━━━━━━━━━━━━━━━━━━━━ 2s 811us/step
1287/1287 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step

=== For Dataset unsw, WOA optimizer ===
Threshold Percentile Used: 40
Calculated Threshold Value: 163739.9838

=== Best Parameters ===
Latent Dim: 53
Neurons/Layer: 128
Learning Rate: 0.007993060372830981
Dropout Rate: 0.5
Batch Size: 256
Epochs: 100
Activation: sigmoid
Output Activation: relu
Loss Function: categorical_crossentropy
Optimizer: SGD

=== Evaluation Metrics ===
Accuracy: 0.5738
Precision: 0.5648
Recall: 0.9608
F1 Score: 0.7114
AUC: 0.4587
Confusion Matrix:
[[ 1997 16663]
 [  882 21624]]


#### ************Autoencoder + Binary Differential Evolution Algorithm************

In [ ]:
main_pipeline('kdd99', 'DE')

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 02:42:59 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 776us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 853us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 837us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 755us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 726us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 746us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 763us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 830us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433

2025/03/27 02:46:22 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: -0.8447802197802198, Global best: -0.8447802197802198, Runtime: 99.43810 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 871us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 749us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 827us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 777us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 780us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 817us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 804us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 751us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 03:24:48 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 19, Current best: -0.9395604395604396, Global best: -0.9395604395604396, Runtime: 162.60767 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 795us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 764us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 810us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 766us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 727us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 03:27:38 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 20, Current best: -0.9395604395604396, Global best: -0.9395604395604396, Runtime: 170.09932 seconds


433/433 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
305/305 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step

=== For Dataset kdd99, DE optimizer ===
Threshold Percentile Used: 60
Calculated Threshold Value: 2918446107055.1709

=== Best Parameters ===
Latent Dim: 38
Neurons/Layer: 100
Learning Rate: 0.007919439264665502
Dropout Rate: 0.2984549616826757
Batch Size: 256
Epochs: 77
Activation: relu
Output Activation: linear
Loss Function: categorical_crossentropy
Optimizer: Adam

=== Evaluation Metrics ===
Accuracy: 0.5563
Precision: 0.0593
Recall: 0.0039
F1 Score: 0.0074
AUC: 0.8254
Confusion Matrix:
[[5398  254]
 [4064   16]]


In [ ]:
main_pipeline('unsw', 'DE')

521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 03:28:12 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 720us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 731us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 734us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/

2025/03/27 03:32:02 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: -0.7297605473204105, Global best: -0.7297605473204105, Runtime: 112.00616 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 722us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 740us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 728us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 725us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 782us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 744us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 733us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 767us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 757us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 765us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 04:01:32 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 19, Current best: -0.7559863169897377, Global best: -0.7559863169897377, Runtime: 124.00140 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 750us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 742us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 772us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 787us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 775us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 738us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 745us/step
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


2025/03/27 04:04:03 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 20, Current best: -0.7559863169897377, Global best: -0.7559863169897377, Runtime: 150.89215 seconds


521/521 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step
258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step

=== For Dataset unsw, DE optimizer ===
Threshold Percentile Used: 40
Calculated Threshold Value: 1461.4598

=== Best Parameters ===
Latent Dim: 17
Neurons/Layer: 96
Learning Rate: 0.004610828298159693
Dropout Rate: 0.45382521469407544
Batch Size: 256
Epochs: 75
Activation: relu
Output Activation: sigmoid
Loss Function: binary_crossentropy
Optimizer: Adam

=== Evaluation Metrics ===
Accuracy: 0.3886
Precision: 0.1084
Recall: 0.0145
F1 Score: 0.0256
AUC: 0.3407
Confusion Matrix:
[[3133  543]
 [4491   66]]


### +++++++++++++++++++++++++++++++++++++++++++++++ **Isolation Forest** +++++++++++++++++++++++++++++++++++++++++++++++

#### ************Architecture************

In [ ]:
def create_isolation_forest(n_estimators, max_samples, contamination, max_features, bootstrap, random_state):
    """
    Create an Isolation Forest model with specified parameters.
    
    Args:
        n_estimators (int): Number of base estimators
        max_samples (int): Number of samples to draw to train each base estimator
        contamination (float): Proportion of outliers in the dataset
        max_features (float): Proportion of features to consider
        bootstrap (bool): Whether samples are drawn with replacement
        random_state (int): Random seed
        
    Returns:
        IsolationForest: Configured Isolation Forest model
    """
    return IsolationForest(
        n_estimators=int(n_estimators),
        max_samples=int(max_samples),
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=int(random_state),
        n_jobs=-1  # Use all available cores
    )

In [ ]:

def objective_function(solution, X_train, X_val, y_val, dataset_name):
    """
    Objective function for optimization - evaluates Isolation Forest performance.
    
    Args:
        solution (list): Parameters to optimize [n_estimators, max_samples, contamination, max_features, bootstrap, random_state]
        X_train (array): Training features
        y_train (array): Training labels (0=normal, 1=anomaly)
        
    Returns:
        float: Fitness value (1 - AUC score to be minimized)
    """
    # Convert solution to parameters
    n_estimators, max_samples, contamination, max_features, bootstrap, random_state = solution
    bootstrap = bool(round(bootstrap))
    
    y_val_binary = np.where(y_val > 0, 1, 0)
    # Create and train model
    model = create_isolation_forest(
        n_estimators=n_estimators,
        max_samples=max_samples,
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=random_state
    )
    

    model.fit(X_train)
    
    scores = -model.decision_function(X_val)

    if dataset_name == "kdd99":
        threshold = np.percentile(scores, 60)
    elif dataset_name == "unsw":
        threshold = np.percentile(scores, 40)
    
    y_pred = (scores > threshold).astype(int)
    
    # Use F1 score as optimization target (or whatever metric you prefer)
    try:
        f1 = f1_score(y_val, y_pred)
        return 1 - f1  # Minimize (1 - F1)
    except ValueError:
        return 1.  # Worst possible score if calculation fails

In [ ]:

def optimize_isolation_forest(dataset_name, X_train, X_val, y_val, optimizer_type, epochs=50, pop_size=20):
    # Function implementation
    """
    Optimize Isolation Forest parameters using specified optimizer.
    
    Args:
        X_train (array): Training features
        y_train (array): Training labels
        optimizer_type (str): 'WOA' or 'DE'
        epochs (int): Number of optimization iterations
        pop_size (int): Population size
        
    Returns:
        list: Best parameters found
    """
    # Define the search space
    problem_dict = {
        "bounds": FloatVar(
            lb=[
                50,     # n_estimators (50-500)
                100,    # max_samples (100-1000)
                0.01,   # contamination (0.01-0.5)
                0.1,    # max_features (0.1-1.0)
                0,      # bootstrap (0=False, 1=True)
                0       # random_state (0-100)
            ],
            ub=[
                500,1000,0.5,1.0,1,100
            ],
            name="delta",
        ),
        "minmax": "min",  # We're minimizing (1 - AUC)
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, y_val, dataset_name),
    }
    
    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=10)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size, miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1)
    else:
        raise ValueError("Optimizer not supported. Choose 'WOA' or 'DE'.")
    
    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution


In [ ]:

def evaluate_model(model, X_test, y_test, dataset_name):
    """
    Evaluate Isolation Forest model performance.
    
    Args:
        model: Trained Isolation Forest model
        X_test (array): Test features
        y_test (array): Test labels (0=normal, 1=anomaly)
        
    Returns:
        tuple: Evaluation metrics (accuracy, precision, recall, f1, auc, confusion_matrix)
    """
    # Get anomaly scores
    scores = -model.decision_function(X_test)  # Convert to positive where higher is more anomalous
    
    # Convert scores to binary predictions using the contamination threshold
    if dataset_name == "kdd99":
        threshold = np.percentile(scores, 60)
    elif dataset_name == "unsw":
        threshold = np.percentile(scores, 40)
    y_pred = (scores > threshold).astype(int)
    
    # Compute Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, scores)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return accuracy, precision, recall, f1, auc, conf_matrix


In [ ]:
def main_pipeline(dataset_name, optimizer_type):
    """
    Main pipeline for optimizing and evaluating Isolation Forest.
    
    Args:
        dataset_name (str): 'kdd99' or 'unsw'
        optimizer_type (str): 'WOA' or 'DE'
    """
    if dataset_name == "kdd99":
        X_train = df_kdd99_train
        X_val = df_kdd99_validation
        y_val = df_kdd99_validation_label
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
    elif dataset_name == "unsw":
        X_train = df_unsw_train
        X_val = df_unsw_validation
        y_val = df_unsw_validation_label
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
    else:
        raise ValueError("Dataset not supported. Choose 'kdd99' or 'unsw'.")
    
    best_params = optimize_isolation_forest(dataset_name, X_train, X_val, y_val, optimizer_type)
        
    # Extract parameters
    (n_estimators, max_samples, contamination, 
     max_features, bootstrap, random_state) = best_params
    
    # Convert binary bootstrap flag to boolean
    bootstrap = bool(round(bootstrap))
    
    # Create final model with best parameters
    model = create_isolation_forest(
        n_estimators=int(n_estimators),
        max_samples=int(max_samples),
        contamination=contamination,
        max_features=max_features,
        bootstrap=bootstrap,
        random_state=int(random_state))
    
    # Train final model
    model.fit(X_train)
    
    # Evaluate on test set
    accuracy, precision, recall, f1, auc, conf_matrix = evaluate_model(model, X_test, y_test, dataset_name)
    
    # Print results
    print(f"\n=== For Dataset {dataset_name}, {optimizer_type} optimizer ===")
    
    print("\n=== Best Parameters ===")
    print(f"n_estimators: {int(n_estimators)}")
    print(f"max_samples: {int(max_samples)}")
    print(f"contamination: {contamination:.4f}")
    print(f"max_features: {max_features:.4f}")
    print(f"bootstrap: {bootstrap}")
    print(f"random_state: {int(random_state)}")
    
    print("\n=== Evaluation Metrics ===")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC: {auc:.4f}")
    print(f"Confusion Matrix:\n{conf_matrix}")



#### ************Isolation Forest + Whale Optimization Algorithm************

In [ ]:
main_pipeline('kdd99', 'WOA')

2025/03/26 08:53:14 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/03/26 08:54:30 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.32554530562979567, Global best: 0.32554530562979567, Runtime: 32.07965 seconds
2025/03/26 08:54:57 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.32554530562979567, Global best: 0.32554530562979567, Runtime: 27.32851 seconds
2025/03/26 08:55:29 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.32554530562979567, Global best: 0.32554530562979567, Runtime: 31.31265 seconds
2025/03/26 08:56:04 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.318130873351151, Global best: 0.318130873351151, Runtime: 34.90811 seconds
2025/03/26 08:56:23 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.318130873351151, Global best: 0.318130873351151, Runtime: 19.63727 sec


=== For Dataset kdd99, WOA optimizer ===

=== Best Parameters ===
n_estimators: 53
max_samples: 142
contamination: 0.1650
max_features: 0.1174
bootstrap: False
random_state: 49

=== Evaluation Metrics ===
Accuracy: 0.9177
Precision: 0.9248
Recall: 0.8761
F1 Score: 0.8998
AUC: 0.9729
Confusion Matrix:
[[53354  2925]
 [ 5087 35959]]


In [ ]:
main_pipeline('unsw', 'WOA')


2025/03/26 09:24:29 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/03/26 09:25:50 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.43402360276107776, Global best: 0.43402360276107776, Runtime: 31.59609 seconds
2025/03/26 09:26:16 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.43402360276107776, Global best: 0.43402360276107776, Runtime: 26.15176 seconds
2025/03/26 09:26:37 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.394477844578045, Global best: 0.394477844578045, Runtime: 20.87440 seconds
2025/03/26 09:26:54 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.394477844578045, Global best: 0.394477844578045, Runtime: 17.24975 seconds
2025/03/26 09:27:14 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.394477844578045, Global best: 0.394477844578045, Runtime: 20.21273 seconds


=== For Dataset unsw, WOA optimizer ===

=== Best Parameters ===
n_estimators: 52
max_samples: 693
contamination: 0.1629
max_features: 0.1040
bootstrap: False
random_state: 2

=== Evaluation Metrics ===
Accuracy: 0.4321
Precision: 0.4856
Recall: 0.5292
F1 Score: 0.5065
AUC: 0.4289
Confusion Matrix:
[[11590 25410]
 [21343 23989]]


#### ************Isolation Forest + Binary Differential Evolution Algorithm************

In [ ]:
main_pipeline('kdd99', 'DE')

2025/03/26 09:06:30 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/03/26 09:07:49 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.36632468316234157, Global best: 0.36632468316234157, Runtime: 41.17496 seconds
2025/03/26 09:08:35 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.3646003965859126, Global best: 0.3646003965859126, Runtime: 45.82095 seconds
2025/03/26 09:09:30 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.36356582464005516, Global best: 0.36356582464005516, Runtime: 55.13234 seconds
2025/03/26 09:10:26 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.3437365290111216, Global best: 0.3437365290111216, Runtime: 55.47235 seconds
2025/03/26 09:11:09 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.3437365290111216, Global best: 0.3437365290111


=== For Dataset kdd99, DE optimizer ===

=== Best Parameters ===
n_estimators: 50
max_samples: 656
contamination: 0.5000
max_features: 0.1000
bootstrap: True
random_state: 74

=== Evaluation Metrics ===
Accuracy: 0.9713
Precision: 0.9914
Recall: 0.9402
F1 Score: 0.9651
AUC: 0.9937
Confusion Matrix:
[[55943   336]
 [ 2455 38591]]


In [ ]:
main_pipeline('unsw', 'DE')

2025/03/26 09:38:40 PM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/03/26 09:40:38 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.4315742596303719, Global best: 0.4315742596303719, Runtime: 61.34091 seconds
2025/03/26 09:41:32 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.4286795813849922, Global best: 0.4286795813849922, Runtime: 54.75517 seconds
2025/03/26 09:42:26 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.4286795813849922, Global best: 0.4286795813849922, Runtime: 53.14702 seconds
2025/03/26 09:43:13 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.4286795813849922, Global best: 0.4286795813849922, Runtime: 47.75386 seconds
2025/03/26 09:44:01 PM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.4286795813849922, Global best: 0.4286795813849922,


=== For Dataset unsw, DE optimizer ===

=== Best Parameters ===
n_estimators: 64
max_samples: 997
contamination: 0.0224
max_features: 0.1201
bootstrap: True
random_state: 100

=== Evaluation Metrics ===
Accuracy: 0.4159
Precision: 0.4721
Recall: 0.5144
F1 Score: 0.4924
AUC: 0.4341
Confusion Matrix:
[[10924 26076]
 [22011 23321]]


### +++++++++++++++++++++++++++++++++++++++++++++++ **DBSCAN** +++++++++++++++++++++++++++++++++++++++++++++++

#### ************Architecture************

In [ ]:
def create_dbscan(eps, min_samples):
    """Create a DBSCAN model with specified parameters."""
    return DBSCAN(
        eps=float(eps),
        min_samples=int(min_samples),
        n_jobs=-1
    )

In [ ]:
def objective_function(solution, X_train, X_val, y_val, dataset_name):
    """Improved objective function using validation set and classification metrics"""
    eps, min_samples = solution
    
    # Create and fit DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=int(min_samples))
    dbscan.fit(X_train)  # Train on training data
    
    # 1. Identify core points from training data
    core_samples_mask = np.zeros_like(dbscan.labels_, dtype=bool)
    core_samples_mask[dbscan.core_sample_indices_] = True
    core_points = X_train[core_samples_mask]
    
    # 2. For each validation point, calculate distance to nearest core point
    if len(core_points) > 0:
        nbrs = NearestNeighbors(n_neighbors=1).fit(core_points)
        distances, _ = nbrs.kneighbors(X_val)
    else:
        distances = np.array([[np.inf]] * len(X_val))
    
    # 3. Set dataset-specific threshold
    if dataset_name == "kdd99":
        percentile = 60
    elif dataset_name == "unsw":
        percentile = 40
    
    if len(core_points) > 0:
        train_distances, _ = nbrs.kneighbors(X_train)
        threshold = np.percentile(train_distances, percentile)
    else:
        threshold = np.inf
    
    # 4. Classify validation points
    y_pred = (distances > threshold).astype(int).flatten()
    
    # Handle degenerate cases
    if len(np.unique(y_pred)) == 1:
        return 1.0  # Worst possible fitness (all same prediction)
    
    # Use F1-score as the optimization metric (can be changed to other metrics)
    return 1 - f1_score(y_val, y_pred)  # Minimize (1 - F1)



In [ ]:
def optimize_dbscan(X_train, X_val, y_val, optimizer_type, dataset_name, epochs=50, pop_size=20):
    problem_dict = {
        "bounds": FloatVar(lb=[0.1, 2], ub=[1.0, 20], name="delta"),
        "minmax": "min",
        "obj_func": lambda solution: objective_function(solution, X_train, X_val, y_val, dataset_name)
    }

    # Choose Optimizer
    if optimizer_type == "WOA":
        model = WOA.HI_WOA(epoch=epochs, pop_size=pop_size, feedback_max=10)
    elif optimizer_type == "DE":
        model = DE.JADE(epoch=epochs, pop_size=pop_size,
                        miu_f=0.5, miu_cr=0.5, pt=0.1, ap=0.1)
    else:
        raise ValueError("Optimizer not supported. Choose 'WOA' or 'DE'.")

    # Run Optimization
    g_best = model.solve(problem_dict)
    return g_best.solution



In [ ]:
def evaluate_model(model, X_train, X_test, y_test, dataset_name):
    """Evaluate DBSCAN model performance on test set using distance-based approach."""
    # 1. Identify core points from training data
    model.fit(X_train)
    core_samples_mask = np.zeros_like(model.labels_, dtype=bool)
    core_samples_mask[model.core_sample_indices_] = True
    core_points = X_train[core_samples_mask]
    
    # 2. For each test point, calculate distance to nearest core point
    if len(core_points) > 0:
        nbrs = NearestNeighbors(n_neighbors=1).fit(core_points)
        distances, _ = nbrs.kneighbors(X_test)
    else:
        distances = np.array([[np.inf]] * len(X_test))
    
    # 3. Set dataset-specific threshold
    if dataset_name == "kdd99":
        percentile = 60
    elif dataset_name == "unsw":
        percentile = 40
    
    if len(core_points) > 0:
        train_distances, _ = nbrs.kneighbors(X_train)
        threshold = np.percentile(train_distances, percentile)
    else:
        threshold = np.inf
    
    # 4. Classify test points
    y_pred = (distances > threshold).astype(int).flatten()
    y_scores = distances.flatten()  # Use distances as scores for AUC
    
    return {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred),
        "recall": recall_score(y_test, y_pred),
        "f1": f1_score(y_test, y_pred),
        "auc": roc_auc_score(y_test, y_scores),
        "confusion_matrix": confusion_matrix(y_test, y_pred)
    }


In [ ]:
def main_pipeline(dataset_name, optimizer_type):
    """Main pipeline with proper train/test split."""
    # Load dataset
    if dataset_name == "kdd99":
        X_train = df_kdd99_training
        X_val = df_kdd99_validation
        y_val = df_kdd99_validation_label
        X_test = df_kdd99_testing
        y_test = df_kdd99_testing_label
    elif dataset_name == "unsw":
        X_train = df_unsw_train
        X_val = df_unsw_validation
        y_val = df_unsw_validation_label
        X_test = df_unsw_testing
        y_test = df_unsw_testing_label
    else:
        raise ValueError("Dataset not supported. Choose 'kdd99' or 'unsw'.")
    
    # Optimize parameters
    best_params = optimize_dbscan(X_train, X_val, y_val, optimizer_type, dataset_name)
    
    # Create final model with best parameters
    eps, min_samples = best_params
    final_model = DBSCAN(eps=eps, min_samples=int(min_samples))
    
    # Evaluate on test set using distance-based approach
    metrics = evaluate_model(final_model, X_train, X_test, y_test, dataset_name)
    
    # Print results
    print(f"\n=== Results for {dataset_name} with {optimizer_type} ===")
    print("\nBest Parameters:")
    print(f"eps: {eps:.4f}")
    print(f"min_samples: {int(min_samples)}")
    
    print("\nTest Set Metrics:")
    print(f"Accuracy: {metrics['accuracy']:.4f}")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall: {metrics['recall']:.4f}")
    print(f"F1 Score: {metrics['f1']:.4f}")
    print(f"AUC: {metrics['auc']:.4f}")
    print("Confusion Matrix:")
    print(metrics["confusion_matrix"])

#### ************DBSCAN + Whale Optimization Algorithm************

In [ ]:
main_pipeline('unsw', 'WOA')


2025/03/26 10:29:57 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/03/26 10:31:07 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.4157303370786517, Global best: 0.4157303370786517, Runtime: 36.53319 seconds
2025/03/26 10:31:42 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.3607355327203894, Global best: 0.3607355327203894, Runtime: 34.50083 seconds
2025/03/26 10:32:18 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.3607355327203894, Global best: 0.3607355327203894, Runtime: 36.34108 seconds
2025/03/26 10:32:53 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.3575932264456855, Global best: 0.3575932264456855, Runtime: 34.63153 seconds
2025/03/26 10:33:29 PM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.3575932264456855, Global best: 0.3575932264456855, Runtime: 35.71510 secon


=== Results for unsw with WOA ===

Best Parameters:
eps: 0.9999
min_samples: 20

Test Set Metrics:
Accuracy: 0.5456
Precision: 0.5463
Recall: 0.9971
F1 Score: 0.7058
AUC: 0.3377
Confusion Matrix:
[[   21 18639]
 [   66 22440]]


In [ ]:
main_pipeline('kdd99', 'WOA')

2025/03/27 12:26:42 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: Solving single objective optimization problem.
2025/03/27 12:32:16 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 1, Current best: 0.9387755102040817, Global best: 0.9387755102040817, Runtime: 176.76551 seconds
2025/03/27 12:35:18 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 2, Current best: 0.9386581469648563, Global best: 0.9386581469648563, Runtime: 181.96774 seconds
2025/03/27 12:38:20 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 3, Current best: 0.9226450405489707, Global best: 0.9226450405489707, Runtime: 182.14333 seconds
2025/03/27 12:41:26 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 4, Current best: 0.9226450405489707, Global best: 0.9226450405489707, Runtime: 185.84573 seconds
2025/03/27 12:44:31 AM, INFO, mealpy.swarm_based.WOA.HI_WOA: >>>Problem: P, Epoch: 5, Current best: 0.9226450405489707, Global best: 0.9226450405489707, Runtime: 184.62949 


=== Results for kdd99 with WOA ===

Best Parameters:
eps: 1.0000
min_samples: 2

Test Set Metrics:
Accuracy: 0.5479
Precision: 0.2158
Recall: 0.0283
F1 Score: 0.0500
AUC: 0.2044
Confusion Matrix:
[[26082  2104]
 [19897   579]]


#### ************DBSCAN + Binary Differential Evolution Algorithm************

In [ ]:
main_pipeline('kdd99', 'DE')

2025/03/27 03:59:07 AM, INFO, mealpy.evolutionary_based.DE.JADE: Solving single objective optimization problem.
2025/03/27 04:04:24 AM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 1, Current best: 0.9944598337950139, Global best: 0.9944598337950139, Runtime: 161.59975 seconds
2025/03/27 04:07:11 AM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 2, Current best: 0.9386581469648563, Global best: 0.9386581469648563, Runtime: 167.17440 seconds
2025/03/27 04:10:10 AM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 3, Current best: 0.9386581469648563, Global best: 0.9386581469648563, Runtime: 178.88860 seconds
2025/03/27 04:13:12 AM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 4, Current best: 0.9386581469648563, Global best: 0.9386581469648563, Runtime: 181.31031 seconds
2025/03/27 04:16:21 AM, INFO, mealpy.evolutionary_based.DE.JADE: >>>Problem: P, Epoch: 5, Current best: 0.9386581469648563, Global best: 0.9386581469648


=== Results for kdd99 with DE ===

Best Parameters:
eps: 0.1000
min_samples: 2

Test Set Metrics:
Accuracy: 0.5545
Precision: 0.0275
Recall: 0.0017
F1 Score: 0.0032
AUC: 0.5339
Confusion Matrix:
[[26947  1239]
 [20441    35]]


In [ ]:
main_pipeline('unsw', 'DE')

c:\Users\DSAIRC 01\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\DSAIRC 01\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\DSAIRC 01\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\DSAIRC 01\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_f


=== Results for unsw with DE ===

Best Parameters:
eps: 1.0000
min_samples: 18

Test Set Metrics:
Accuracy: 0.5467
Precision: 0.5467
Recall: 1.0000
F1 Score: 0.7069
AUC: 0.3403
Confusion Matrix:
[[    0 18660]
 [    0 22506]]
